# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,flying fish cove,-10.4217,105.6791,23.82,94,100,3.60,CX,1692123708
1,1,sitka,57.0531,-135.3300,13.94,88,100,5.66,US,1692123709
2,2,keflavik,64.0049,-22.5624,13.24,62,75,5.14,IS,1692123710
3,3,waitangi,-43.9535,-176.5597,9.34,96,100,3.52,NZ,1692123710
4,4,ginir,7.1333,40.7000,19.43,51,22,2.53,ET,1692123711


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()
# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,pacific grove,36.6177,-121.9166,26.33,83,0,3.09,US,1692123587
57,57,gazojak,41.1875,61.4036,25.07,16,0,4.21,TM,1692123720
136,136,la sarre,48.8002,-79.1996,22.35,31,0,2.15,CA,1692123754
137,137,ribeira grande,38.5167,-28.7000,25.43,76,0,0.89,PT,1692123755
162,162,ghormach,35.7306,63.7826,21.75,15,0,1.44,AF,1692123760
221,221,brookings,44.3114,-96.7984,25.59,64,0,3.60,US,1692123695
325,325,horta,38.5333,-28.6333,25.01,76,0,0.89,PT,1692123811
368,368,saint-denis,48.9167,2.3333,25.59,58,0,2.57,FR,1692123825
381,381,shevchenko,48.5000,35.0000,26.26,45,0,2.85,UA,1692123828
389,389,sutysky,49.0425,28.4173,25.50,29,0,2.95,UA,1692123830


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
29,pacific grove,US,36.6177,-121.9166,83,
57,gazojak,TM,41.1875,61.4036,16,
136,la sarre,CA,48.8002,-79.1996,31,
137,ribeira grande,PT,38.5167,-28.7000,76,
162,ghormach,AF,35.7306,63.7826,15,
221,brookings,US,44.3114,-96.7984,64,
325,horta,PT,38.5333,-28.6333,76,
368,saint-denis,FR,48.9167,2.3333,58,
381,shevchenko,UA,48.5000,35.0000,45,
389,sutysky,UA,49.0425,28.4173,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
import pprint
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"

    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
#     pprint.pprint(name_address)
#     break
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pacific grove - nearest hotel: Pacific Grove Inn
gazojak - nearest hotel: Жайхун
la sarre - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
ghormach - nearest hotel: No hotel found
brookings - nearest hotel: Quality Inn
horta - nearest hotel: Azoris Faial Garden - Resort Hotel
saint-denis - nearest hotel: Residhome
shevchenko - nearest hotel: Premier Hotel Abri
sutysky - nearest hotel: No hotel found
vitkov - nearest hotel: Růže
colonia - nearest hotel: Wasserturm Hotel Cologne
kolonia - nearest hotel: Wasserturm Hotel Cologne
shizilu - nearest hotel: No hotel found
jaguaruna - nearest hotel: No hotel found
quelimane - nearest hotel: Mabassa


,City,Country,Lat,Lng,Humidity,Hotel Name
29,pacific grove,US,36.6177,-121.9166,83,Pacific Grove Inn
57,gazojak,TM,41.1875,61.4036,16,Жайхун
136,la sarre,CA,48.8002,-79.1996,31,No hotel found
137,ribeira grande,PT,38.5167,-28.7000,76,Hospedaria JSF
162,ghormach,AF,35.7306,63.7826,15,No hotel found
221,brookings,US,44.3114,-96.7984,64,Quality Inn
325,horta,PT,38.5333,-28.6333,76,Azoris Faial Garden - Resort Hotel
368,saint-denis,FR,48.9167,2.3333,58,Residhome
381,shevchenko,UA,48.5000,35.0000,45,Premier Hotel Abri
389,sutysky,UA,49.0425,28.4173,29,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)



# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)